In [ ]:
# score 정보 -> (원하는 것) 어떤 문장이 몇점인지를 예측 -> 이 값이 평균 이상이면 긍정, 평균 미만이면 부정
#            -> 긍/부정 으로만 학습하면 결과가 더 부정확할 수도. 
#            -> (현행) 평균 이상이면 100, 미만 0으로 분류해서 학습, 적용

# kkm 형태소 분석기에서 사전 조정해서, 칼퇴, 워라밸 같은 우리가 사용하는 단어로 수정하기.
# raw data : 맞춤법 교정.
# 조사 같은 연결어 제외하기. feature 추가(전전단어와 후후단어)
# CRF : N-gram ?????  kkm 사전을 형태소로 나눠놓고 하는 걸로. 

# 긍정/부정 리뷰 스크래핑+키워드 추출+사전으로 추가(feature) 
# 명사(를 비롯한 주요 단어)인 pos tag 만 뽑아서 feature에서 가중치 주기. 
# feature: pos tag에서 규칙 찾아서(eg. NNG+NNG) 여러 단어 연접하여 새롭게 긍/부정어 사전 돌리기 ??? <- 복잡. 
# 쪼개진 긍부정어 사전(만들어진 txt) 찾아보기.

# 조사 없애기? pos tag 포기. 띄어쓰기 교정 + 맞춤법 교정 + CRF ...? 
# feat -1, +1 < 최고로 나쁘다 -> 최고 로 나쁘 다 : 이런 식으로 CRF 특징이 무효화 됨.  

# CRF가 아니라, 다른 방식으로 해보는 게 어떤지? eg. review classification처럼 
# summarization + 빈도수 처리 + .... + N-gram 활용
# 시각화: eli5 

In [7]:
from pykospacing import Spacing
import requests
import time
import re
import pandas as pd
import pprint
import pycrfsuite
import eli5
#import lime
#from lime.lime_text import LimeTextExplainer

from sklearn_crfsuite import CRF 
from sklearn.model_selection import cross_val_predict # train_test_split <- 쓰려면 쓰기
from sklearn_crfsuite .metrics import flat_classification_report # metrics: measure(Accuracy, score 등) 도구
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import sent_to_xy
from bs4 import BeautifulSoup
from konlpy.tag import Kkma as kkm
from konlpy.tag import Okt
#from konlpy.tag import Mecab
#import Mecab
import tweepy
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict

# 기계 학습
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline

In [8]:
# 로그인 할 url
url = "https://www.jobplanet.co.kr/users/sign_in?_nav=gb"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'
headers = {'Content-type': 'application/json', 'Accept': 'text/plain', 'User-Agent':user_agent}
login_data = {'user':{'email':'hsmy31@hanyang.ac.kr', 'password':'rhkwpgksmswnd!', 'remember_me':'true'}}
session = requests.session()

# 로그인 실행
login_response = session.post(url, json = login_data, headers = headers)

reviews = []

In [9]:
def ind_reviews(code):
    url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code)
    response =  session.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    num = soup.find('span', class_='num') # 리뷰 개수 확인
    num = int(num.get_text().strip())
    import math, random
    pages = math.ceil(num / 10)
    page = random.sample(range(1, pages),30)
    reviews = {}
    for i in page: # 페이지를 랜덤으로 하여 300개의 리뷰 추출
        time.sleep(1)
        url = "https://www.jobplanet.co.kr/reviews?&industry_id=" + str(code) + "&page="+ str(i)
        response =  session.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        label = soup.find_all('h2', class_="us_label") # 한 줄 리뷰 추출
        # 추출한 리뷰에서 태그를 제외한 텍스트만 추출하여 labels 리스트에 저장
        labels = [label.get_text().strip() for label in label[:]]
        star = soup.find_all('div', class_="star_score") # 별점이 포함된 div 추출
        # 추출한 div에서 별점을 나타내는 width 속성의 숫자로 된 부분을 stars 리스트에 저장
        stars = re.findall('[0-9]+[.]+[0-9]', str(star))
        # reviews 딕셔너리에 '리뷰: 별점' 형식으로 추가
        for j in range(len(labels)):
            reviews[labels[j][5:-1]] = stars[j].replace('.0','')
    return reviews

In [ ]:
dict_pos=[]
dict_neg=[]
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/pos_pol_word.txt", 'rt', encoding='UTF8'):
    dict_pos.append(line.split('\n')[0])
    
for line in open("C:/MY/대학/3학년 2학기/텍스트마이닝/final project/word_dict/neg_pol_word.txt", 'rt', encoding='UTF8'):
    dict_neg.append(line.split('\n')[0])

In [10]:
# 각 산업군의 기업 리뷰 스크래핑
reviews1=dict(ind_reviews(1001))

In [11]:
reviews1

{'안정적으로 꾸준히 오래 다닐 수 있는 회사, 그러나 조금 고여있는 회사': '80',
 '워라밸의 끝판왕을 경험할 수 있으나 미국문화에 익숙하지 않으면 위로 올라가기 어려운 곳.': '80',
 '일은 많고 열심히 안하는 직원도 있지만 친절하고 배울만했습니다. 하지만 민원인이 많을 때는 밉기도 했네요': '80',
 '공기좋고 선진국인 호주에서도 수준높은 도시인 캔버라에서 5시 칼퇴가 가능한 직장': '80',
 '중앙부처 중에 그래도 편안하게 일 할 수 있는 조직 분위기': '80',
 '30년전 뒤쳐진 조직 문화, 업무프로세스, 왜 공무원인가?': '80',
 '지방공기업으로 정년보장됩니다.복지포인트도 지급됩니다.': '80',
 '열심히 노력하여 취업한만큼 보람이있는 직장.': '80',
 '공공기관이고, 경찰관, 행정관, 무기계약직으로 이루어진 공무원조직입니다. 대략 680여명 근무중이고 내근, 외근 으로 이루어져 있습니다.': '80',
 '계속 소통을 놓치지 않으려고 하며 수평적입니다. 직원들이 편안하게 일할 수 있는 분위기가 형성되어 있습니다.': '80',
 '정년보장, 칼퇴근, 적은급여, 껍데기는 공기업\r이상하게 직원들이 공기업 다닌다고 자부심\r가지고 있음 중소기업급임': '76',
 '팀바이팀, 자부심은 많지만 월급은 적고 브레인은 많지만 인재를 제대로 활용하지 못하는 곳.': '76',
 '안정적인 직장이나 그만큼 폐쇄적인 조직': '76',
 '최소한의 사회적 관계에서 본인 업무를 할 수 있는 공무원': '76',
 '부대에서 근무할 때는 편한 줄 모름. 하지만 사회에 나오면 이 곳보다 편한 곳이 없음. 야근, 회식 강요 없고 칼퇴근 가능. 개인주의라 업무만 잘 하면 그 외의 것으로 상사 눈치도 별로 안봐도 됨.': '76',
 '서울 고정 괜찮은 급여 미친듯한 워라밸 한량들이 다니기 딱좋은 회사.': '76',
 '극지 방문이라는 평생 해보기 힘든 경험을 할 기회가 있음.': '76',
 '노동자의 권리 향상을 위해 힘쓰는 곳':

In [12]:
text1=list(reviews1.keys())
text2=[]
for t in text1:
     text2.append(re.sub('\r',' ', t))
score1=list(reviews1.values())

In [13]:
sum=0
for i in range(len(score1)):
    sum+=int(score1[i])
avr=sum/len(score1)
print("score 평균값: ", round(avr,3))

score 평균값:  59.52


In [13]:
# kkm 사용
sent_num=0
text1_sent=[]
kkm=kkm()
#mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
for i in range(len(text1)):
    text1_pos=kkm.pos(text1[i])
    sent_num+=1
    for t in text1_pos:
        n=list(t)
        
        sent_number="Sentence: {}".format(sent_num)
        n.insert(0, sent_number)

        if float(score1[(sent_num-1)]) > avr:
            n.insert(3, '100')
        else: 
            n.insert(3, '0')
        
        #n.insert(5, score1[(sent_num-1)])
        
        text1_sent.append(n)
    
pprint.pprint(text1_sent)

[['Sentence: 1', '정년', 'NNG', '100'],
 ['Sentence: 1', '보장', 'NNG', '100'],
 ['Sentence: 1', ',', 'SP', '100'],
 ['Sentence: 1', '칼', 'NNG', '100'],
 ['Sentence: 1', '퇴근', 'NNG', '100'],
 ['Sentence: 1', ',', 'SP', '100'],
 ['Sentence: 1', '적', 'NNG', '100'],
 ['Sentence: 1', '은', 'JX', '100'],
 ['Sentence: 1', '급여', 'NNG', '100'],
 ['Sentence: 1', ',', 'SP', '100'],
 ['Sentence: 1', '껍데기', 'NNG', '100'],
 ['Sentence: 1', '는', 'JX', '100'],
 ['Sentence: 1', '공기업', 'NNG', '100'],
 ['Sentence: 1', '이상', 'NNG', '100'],
 ['Sentence: 1', '하', 'XSV', '100'],
 ['Sentence: 1', '게', 'ECD', '100'],
 ['Sentence: 1', '직원', 'NNG', '100'],
 ['Sentence: 1', '들', 'XSN', '100'],
 ['Sentence: 1', '이', 'JKS', '100'],
 ['Sentence: 1', '공기업', 'NNG', '100'],
 ['Sentence: 1', '다니', 'VV', '100'],
 ['Sentence: 1', 'ㄴ다고', 'ECE', '100'],
 ['Sentence: 1', '자부심', 'NNG', '100'],
 ['Sentence: 1', '가지', 'VV', '100'],
 ['Sentence: 1', '고', 'ECE', '100'],
 ['Sentence: 1', '있음', 'NNG', '100'],
 ['Sentence: 1', '중소기업', '

 ['Sentence: 17', '한데', 'MAG', '0'],
 ['Sentence: 17', '어우러지', 'VV', '0'],
 ['Sentence: 17', 'ㄴ', 'ETD', '0'],
 ['Sentence: 17', '동종', 'NNG', '0'],
 ['Sentence: 17', '업계', 'NNG', '0'],
 ['Sentence: 17', '대기업', 'NNG', '0'],
 ['Sentence: 17', '취급', 'NNG', '0'],
 ['Sentence: 17', '.', 'SF', '0'],
 ['Sentence: 17', '선진적', 'NNG', '0'],
 ['Sentence: 17', '이', 'VCP', '0'],
 ['Sentence: 17', 'ㄴ', 'ETD', '0'],
 ['Sentence: 17', '부분', 'NNG', '0'],
 ['Sentence: 17', '이', 'JKS', '0'],
 ['Sentence: 17', '분명', 'MAG', '0'],
 ['Sentence: 17', '존재', 'NNG', '0'],
 ['Sentence: 17', '하', 'XSV', '0'],
 ['Sentence: 17', 'ㅁ', 'ETN', '0'],
 ['Sentence: 17', '.', 'SF', '0'],
 ['Sentence: 18', '좋', 'VA', '0'],
 ['Sentence: 18', '은', 'ETD', '0'],
 ['Sentence: 18', '사람', 'NNG', '0'],
 ['Sentence: 18', '이', 'JKS', '0'],
 ['Sentence: 18', '많', 'VA', '0'],
 ['Sentence: 18', '지만', 'ECE', '0'],
 ['Sentence: 18', ',', 'SP', '0'],
 ['Sentence: 18', '직원', 'NNG', '0'],
 ['Sentence: 18', '개별', 'NNG', '0'],
 ['Sentence: 18'

 ['Sentence: 31', 'ㄹ', 'ETD', '0'],
 ['Sentence: 31', '때', 'NNG', '0'],
 ['Sentence: 31', '있', 'VV', '0'],
 ['Sentence: 31', '었', 'EPT', '0'],
 ['Sentence: 31', '더', 'EPT', '0'],
 ['Sentence: 31', 'ㄴ', 'ETD', '0'],
 ['Sentence: 31', '직', 'NNG', '0'],
 ['Sentence: 31', '우원', 'NNP', '0'],
 ['Sentence: 31', '들', 'XSN', '0'],
 ['Sentence: 31', '이', 'JKS', '0'],
 ['Sentence: 31', '계속', 'NNG', '0'],
 ['Sentence: 31', '있', 'VV', '0'],
 ['Sentence: 31', '으니', 'ECD', '0'],
 ['Sentence: 31', '...', 'SE', '0'],
 ['Sentence: 31', '발전', 'NNG', '0'],
 ['Sentence: 31', '되', 'XSV', '0'],
 ['Sentence: 31', '지', 'ECD', '0'],
 ['Sentence: 31', '않', 'VXV', '0'],
 ['Sentence: 31', '고', 'ECE', '0'],
 ['Sentence: 31', '항상', 'MAG', '0'],
 ['Sentence: 31', '밑돌', 'VV', '0'],
 ['Sentence: 31', '는', 'ETD', '0'],
 ['Sentence: 31', '듯하', 'VXA', '0'],
 ['Sentence: 31', 'ㄴ', 'ETD', '0'],
 ['Sentence: 31', '느낌', 'NNG', '0'],
 ['Sentence: 31', '이', 'JKS', '0'],
 ['Sentence: 31', '강하', 'VA', '0'],
 ['Sentence: 31', '다',

 ['Sentence: 48', '는', 'ETD', '0'],
 ['Sentence: 48', '곳', 'NNG', '0'],
 ['Sentence: 49', '대한민국', 'NNG', '0'],
 ['Sentence: 49', '창작', 'NNG', '0'],
 ['Sentence: 49', '공연', 'NNG', '0'],
 ['Sentence: 49', '예술', 'NNG', '0'],
 ['Sentence: 49', '에', 'JKM', '0'],
 ['Sentence: 49', '관심', 'NNG', '0'],
 ['Sentence: 49', '이', 'JKS', '0'],
 ['Sentence: 49', '있', 'VA', '0'],
 ['Sentence: 49', '다면', 'ECE', '0'],
 ['Sentence: 49', '다니', 'VV', '0'],
 ['Sentence: 49', '어', 'ECS', '0'],
 ['Sentence: 49', '볼만', 'NNG', '0'],
 ['Sentence: 49', '하', 'XSV', '0'],
 ['Sentence: 49', 'ㄴ', 'ETD', '0'],
 ['Sentence: 49', '회사', 'NNG', '0'],
 ['Sentence: 49', '이', 'VCP', '0'],
 ['Sentence: 49', 'ㅂ니다', 'EFN', '0'],
 ['Sentence: 49', '.', 'SF', '0'],
 ['Sentence: 50', '지역', 'NNG', '0'],
 ['Sentence: 50', '의', 'JKG', '0'],
 ['Sentence: 50', '고인물', 'NNG', '0'],
 ['Sentence: 50', '대잔치', 'NNG', '0'],
 ['Sentence: 50', '와', 'JKM', '0'],
 ['Sentence: 50', '뒷담', 'NNG', '0'],
 ['Sentence: 50', '화', 'XSN', '0'],
 ['Sentence:

 ['Sentence: 69', '도', 'JX', '0'],
 ['Sentence: 69', '사라지', 'VV', '0'],
 ['Sentence: 69', '게', 'ECD', '0'],
 ['Sentence: 69', '함', 'NNG', '0'],
 ['Sentence: 69', '열심히', 'MAG', '0'],
 ['Sentence: 69', '일하', 'VV', '0'],
 ['Sentence: 69', '는', 'ETD', '0'],
 ['Sentence: 69', '데', 'NNB', '0'],
 ['Sentence: 69', '에', 'JKM', '0'],
 ['Sentence: 69', '보람', 'NNG', '0'],
 ['Sentence: 69', '이', 'JKS', '0'],
 ['Sentence: 69', '없', 'VA', '0'],
 ['Sentence: 69', '음', 'ETN', '0'],
 ['Sentence: 70', '경영', 'NNG', '0'],
 ['Sentence: 70', '평가', 'NNG', '0'],
 ['Sentence: 70', '만', 'JX', '0'],
 ['Sentence: 70', '잘', 'MAG', '0'],
 ['Sentence: 70', '받', 'VV', '0'],
 ['Sentence: 70', '으면', 'ECD', '0'],
 ['Sentence: 70', '되', 'VV', '0'],
 ['Sentence: 70', 'ㄴ다는', 'ETD', '0'],
 ['Sentence: 70', '생각', 'NNG', '0'],
 ['Sentence: 70', '으로', 'JKM', '0'],
 ['Sentence: 70', '낮', 'VA', '0'],
 ['Sentence: 70', '은', 'ETD', '0'],
 ['Sentence: 70', '자', 'NNB', '0'],
 ['Sentence: 70', '는', 'JX', '0'],
 ['Sentence: 70', '더', '

...

 ['Sentence: 214', '가', 'JKS', '0'],
 ['Sentence: 214', '심각', 'XR', '0'],
 ['Sentence: 214', '하', 'XSA', '0'],
 ['Sentence: 214', 'ㅁ', 'ETN', '0'],
 ['Sentence: 214', '능력', 'NNG', '0'],
 ['Sentence: 214', '없', 'VA', '0'],
 ['Sentence: 214', '는', 'ETD', '0'],
 ['Sentence: 214', '나이', 'NNG', '0'],
 ['Sentence: 214', '많', 'VA', '0'],
 ['Sentence: 214', '은', 'ETD', '0'],
 ['Sentence: 214', '직원', 'NNG', '0'],
 ['Sentence: 214', '들', 'XSN', '0'],
 ['Sentence: 214', '구조', 'NNG', '0'],
 ['Sentence: 214', '조정', 'NNG', '0'],
 ['Sentence: 214', '필요', 'NNG', '0'],
 ['Sentence: 214', '하', 'XSV', '0'],
 ['Sentence: 214', 'ㅁ', 'ETN', '0'],
 ['Sentence: 214', '노', 'VV', '0'],
 ['Sentence: 214', '는', 'ETD', '0'],
 ['Sentence: 214', '직원', 'NNG', '0'],
 ['Sentence: 214', '들', 'XSN', '0'],
 ['Sentence: 214', '이', 'JKS', '0'],
 ['Sentence: 214', '은근히', 'MAG', '0'],
 ['Sentence: 214', '있음', 'NNG', '0'],
 ['Sentence: 215', '당신', 'NP', '0'],
 ['Sentence: 215', '뒤', 'NNG', '0'],
 ['Sentence: 215', '에', 'JKM', 

 ['Sentence: 235', ',', 'SP', '100'],
 ['Sentence: 235', '같', 'VA', '100'],
 ['Sentence: 235', '은', 'ETD', '100'],
 ['Sentence: 235', '직종', 'NNG', '100'],
 ['Sentence: 235', '회사', 'NNG', '100'],
 ['Sentence: 235', '이직', 'NNG', '100'],
 ['Sentence: 235', '시', 'NNG', '100'],
 ['Sentence: 235', '유니', 'NNP', '100'],
 ['Sentence: 235', '크한', 'UN', '100'],
 ['Sentence: 235', '이력', 'NNG', '100'],
 ['Sentence: 235', '이', 'JKC', '100'],
 ['Sentence: 235', '되', 'VV', '100'],
 ['Sentence: 235', 'ㄹ', 'ETD', '100'],
 ['Sentence: 235', '수', 'NNB', '100'],
 ['Sentence: 235', '있', 'VV', '100'],
 ['Sentence: 235', '습니다', 'EFN', '100'],
 ['Sentence: 235', '.', 'SF', '100'],
 ['Sentence: 236', '혼자', 'NNG', '100'],
 ['Sentence: 236', '만', 'JX', '100'],
 ['Sentence: 236', '의', 'JKG', '100'],
 ['Sentence: 236', '시간', 'NNG', '100'],
 ['Sentence: 236', '을', 'JKO', '100'],
 ['Sentence: 236', '잘', 'MAG', '100'],
 ['Sentence: 236', '관리', 'NNG', '100'],
 ['Sentence: 236', '하', 'XSV', '100'],
 ['Sentence: 236', 'ㄹ

 ['Sentence: 251', '어서', 'ECD', '0'],
 ['Sentence: 251', '좋', 'VA', '0'],
 ['Sentence: 251', '은', 'ETD', '0'],
 ['Sentence: 251', '경험', 'NNG', '0'],
 ['Sentence: 251', '이', 'VCP', '0'],
 ['Sentence: 251', '었', 'EPT', '0'],
 ['Sentence: 251', '습니다', 'EFN', '0'],
 ['Sentence: 251', '.', 'SF', '0'],
 ['Sentence: 252', '전', 'NNG', '0'],
 ['Sentence: 252', '북', 'NNG', '0'],
 ['Sentence: 252', '권', 'XSN', '0'],
 ['Sentence: 252', '에서', 'JKM', '0'],
 ['Sentence: 252', '는', 'JX', '0'],
 ['Sentence: 252', '나름', 'NNG', '0'],
 ['Sentence: 252', '전망', 'NNG', '0'],
 ['Sentence: 252', '있', 'VV', '0'],
 ['Sentence: 252', '고', 'ECE', '0'],
 ['Sentence: 252', '괜찮', 'VA', '0'],
 ['Sentence: 252', '은', 'ETD', '0'],
 ['Sentence: 252', '기업', 'NNG', '0'],
 ['Sentence: 253', '일하', 'VV', '0'],
 ['Sentence: 253', '는', 'ETD', '0'],
 ['Sentence: 253', '사람', 'NNG', '0'],
 ['Sentence: 253', '만', 'JX', '0'],
 ['Sentence: 253', '일하', 'VV', '0'],
 ['Sentence: 253', '는', 'ETD', '0'],
 ['Sentence: 253', '곳', 'NNG', '0'

 ['Sentence: 269', '하', 'XSV', '0'],
 ['Sentence: 269', 'ㄴ', 'ETD', '0'],
 ['Sentence: 269', '편', 'NNB', '0'],
 ['Sentence: 269', '이', 'VCP', '0'],
 ['Sentence: 269', 'ㅂ니다', 'EFN', '0'],
 ['Sentence: 269', '.', 'SF', '0'],
 ['Sentence: 269', '복지', 'NNG', '0'],
 ['Sentence: 269', '라고', 'JX', '0'],
 ['Sentence: 269', '하', 'VV', '0'],
 ['Sentence: 269', 'ㄹ', 'ETD', '0'],
 ['Sentence: 269', '것', 'NNB', '0'],
 ['Sentence: 269', '은', 'JX', '0'],
 ['Sentence: 269', '15', 'NR', '0'],
 ['Sentence: 269', '일', 'NNM', '0'],
 ['Sentence: 269', '연차', 'NNG', '0'],
 ['Sentence: 269', '휴가', 'NNG', '0'],
 ['Sentence: 269', '중', 'NNB', '0'],
 ['Sentence: 269', '8', 'NR', '0'],
 ['Sentence: 269', '일', 'NNM', '0'],
 ['Sentence: 269', '유급', 'NNG', '0'],
 ['Sentence: 269', '휴가', 'NNG', '0'],
 ['Sentence: 269', ',', 'SP', '0'],
 ['Sentence: 269', '야근', 'NNG', '0'],
 ['Sentence: 269', '수당', 'NNG', '0'],
 ['Sentence: 269', '없', 'VA', '0'],
 ['Sentence: 269', '고', 'ECE', '0'],
 ['Sentence: 269', ',', 'SP', '0'],

 ['Sentence: 287', '응대', 'NNG', '0'],
 ['Sentence: 287', '는', 'JX', '0'],
 ['Sentence: 287', '편하', 'VA', '0'],
 ['Sentence: 287', 'ㅁ', 'ETN', '0'],
 ['Sentence: 287', '.', 'SF', '0'],
 ['Sentence: 287', '이것', 'NP', '0'],
 ['Sentence: 287', '이', 'JKS', '0'],
 ['Sentence: 287', '장점', 'NNG', '0'],
 ['Sentence: 287', '다임', 'NNG', '0'],
 ['Sentence: 287', '.', 'SF', '0'],
 ['Sentence: 288', '안정적', 'NNG', '0'],
 ['Sentence: 288', '이', 'VCP', '0'],
 ['Sentence: 288', 'ㄴ', 'ETD', '0'],
 ['Sentence: 288', '공공', 'NNG', '0'],
 ['Sentence: 288', '기관', 'NNG', '0'],
 ['Sentence: 288', '.', 'SF', '0'],
 ['Sentence: 288', '퇴사', 'NNG', '0'],
 ['Sentence: 288', '하', 'XSA', '0'],
 ['Sentence: 288', 'ㄴ지', 'ECD', '0'],
 ['Sentence: 288', '조금', 'MAG', '0'],
 ['Sentence: 288', '되', 'VV', '0'],
 ['Sentence: 288', '었', 'EPT', '0'],
 ['Sentence: 288', '는데', 'ECD', '0'],
 ['Sentence: 288', '그때', 'NNG', '0'],
 ['Sentence: 288', '는', 'JX', '0'],
 ['Sentence: 288', '괜찮', 'VA', '0'],
 ['Sentence: 288', '았', 'EPT', '

In [14]:
from hanspell import spell_checker

In [17]:
# PyKoSpacing(띄어쓰기 교정) + Py-Hanspell(맞춤법 교정) 적용
test2_sent_num=0
test2_text1_sent=[]
spacing = Spacing()
#mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic")
text2_sent=[]
kkm=kkm()

for i in range(len(text2)):
    spelled_sent = spell_checker.check(text2[i])
    hanspell_sent = spelled_sent.checked
    spacing_sent = spacing(hanspell_sent)
    test2_text1_sent.append(spacing_sent)
    test2_sent_num+=1
    text2_pos=kkm.pos(text2[i])
    
    for t in text2_pos:
        n=list(t)
        
        sent_number="Sentence: {}".format(test2_sent_num)
        n.insert(0, sent_number)

        n.insert(3, score1[(test2_sent_num-1)])
        #n.insert(5, score1[(sent_num-1)])
        
        text2_sent.append(n)    

pprint.pprint(text2_sent)

[['Sentence: 1', '안정적', 'NNG', '80'],
 ['Sentence: 1', '으로', 'JKM', '80'],
 ['Sentence: 1', '꾸준히', 'MAG', '80'],
 ['Sentence: 1', '오래', 'MAG', '80'],
 ['Sentence: 1', '다니', 'VV', '80'],
 ['Sentence: 1', 'ㄹ', 'ETD', '80'],
 ['Sentence: 1', '수', 'NNB', '80'],
 ['Sentence: 1', '있', 'VV', '80'],
 ['Sentence: 1', '는', 'ETD', '80'],
 ['Sentence: 1', '회사', 'NNG', '80'],
 ['Sentence: 1', ',', 'SP', '80'],
 ['Sentence: 1', '그러나', 'MAC', '80'],
 ['Sentence: 1', '조금', 'MAG', '80'],
 ['Sentence: 1', '고이', 'VV', '80'],
 ['Sentence: 1', '어', 'ECS', '80'],
 ['Sentence: 1', '있', 'VXV', '80'],
 ['Sentence: 1', '는', 'ETD', '80'],
 ['Sentence: 1', '회사', 'NNG', '80'],
 ['Sentence: 2', '워라밸', 'UN', '80'],
 ['Sentence: 2', '의', 'JKG', '80'],
 ['Sentence: 2', '끝판', 'NNG', '80'],
 ['Sentence: 2', '왕', 'NNG', '80'],
 ['Sentence: 2', '을', 'JKO', '80'],
 ['Sentence: 2', '경험', 'NNG', '80'],
 ['Sentence: 2', '하', 'XSV', '80'],
 ['Sentence: 2', 'ㄹ', 'ETD', '80'],
 ['Sentence: 2', '수', 'NNB', '80'],
 ['Sentence: 2',

 ['Sentence: 10', '있', 'VV', '80'],
 ['Sentence: 10', '는', 'ETD', '80'],
 ['Sentence: 10', '분위기', 'NNG', '80'],
 ['Sentence: 10', '가', 'JKS', '80'],
 ['Sentence: 10', '형성', 'NNG', '80'],
 ['Sentence: 10', '되', 'XSV', '80'],
 ['Sentence: 10', '어', 'ECD', '80'],
 ['Sentence: 10', '있', 'VXV', '80'],
 ['Sentence: 10', '습니다', 'EFN', '80'],
 ['Sentence: 10', '.', 'SF', '80'],
 ['Sentence: 11', '정년', 'NNG', '76'],
 ['Sentence: 11', '보장', 'NNG', '76'],
 ['Sentence: 11', ',', 'SP', '76'],
 ['Sentence: 11', '칼', 'NNG', '76'],
 ['Sentence: 11', '퇴근', 'NNG', '76'],
 ['Sentence: 11', ',', 'SP', '76'],
 ['Sentence: 11', '적', 'NNG', '76'],
 ['Sentence: 11', '은', 'JX', '76'],
 ['Sentence: 11', '급여', 'NNG', '76'],
 ['Sentence: 11', ',', 'SP', '76'],
 ['Sentence: 11', '껍데기', 'NNG', '76'],
 ['Sentence: 11', '는', 'JX', '76'],
 ['Sentence: 11', '공기업', 'NNG', '76'],
 ['Sentence: 11', '이상', 'NNG', '76'],
 ['Sentence: 11', '하', 'XSV', '76'],
 ['Sentence: 11', '게', 'ECD', '76'],
 ['Sentence: 11', '직원', 'NNG', 

 ['Sentence: 19', '공기업', 'NNG', '76'],
 ['Sentence: 19', '의', 'JKG', '76'],
 ['Sentence: 19', '특성', 'NNG', '76'],
 ['Sentence: 19', '상', 'NNG', '76'],
 ['Sentence: 19', '공무원', 'NNG', '76'],
 ['Sentence: 19', '에', 'JKM', '76'],
 ['Sentence: 19', '준하', 'VV', '76'],
 ['Sentence: 19', '는', 'ETD', '76'],
 ['Sentence: 19', '업무', 'NNG', '76'],
 ['Sentence: 19', '안정성', 'NNG', '76'],
 ['Sentence: 19', '을', 'JKO', '76'],
 ['Sentence: 19', '확보', 'NNG', '76'],
 ['Sentence: 19', '하', 'XSV', '76'],
 ['Sentence: 19', 'ㄹ', 'ETD', '76'],
 ['Sentence: 19', '수', 'NNB', '76'],
 ['Sentence: 19', '있음', 'NNG', '76'],
 ['Sentence: 19', '.', 'SF', '76'],
 ['Sentence: 20', '제주도', 'NNP', '76'],
 ['Sentence: 20', '에서', 'JKM', '76'],
 ['Sentence: 20', '는', 'JX', '76'],
 ['Sentence: 20', '그나마', 'MAG', '76'],
 ['Sentence: 20', '가장', 'MAG', '76'],
 ['Sentence: 20', '먹', 'VV', '76'],
 ['Sentence: 20', '고', 'ECE', '76'],
 ['Sentence: 20', '살', 'VV', '76'],
 ['Sentence: 20', '기에', 'ECD', '76'],
 ['Sentence: 20', '적합', '

 ['Sentence: 24', '장난감', 'NNG', '60'],
 ['Sentence: 24', '도', 'JX', '60'],
 ['Sentence: 24', '서관', 'NNG', '60'],
 ['Sentence: 24', '등', 'NNB', '60'],
 ['Sentence: 24', '공공시설', 'NNG', '60'],
 ['Sentence: 24', '위탁', 'NNG', '60'],
 ['Sentence: 24', '운영', 'NNG', '60'],
 ['Sentence: 24', '기업', 'NNG', '60'],
 ['Sentence: 24', '으로', 'JKM', '60'],
 ['Sentence: 24', '서비스', 'NNG', '60'],
 ['Sentence: 24', '기반', 'NNG', '60'],
 ['Sentence: 24', '의', 'JKG', '60'],
 ['Sentence: 24', '구', 'NNG', '60'],
 ['Sentence: 24', '민', 'NNG', '60'],
 ['Sentence: 24', '편의', 'NNG', '60'],
 ['Sentence: 24', '제공', 'NNG', '60'],
 ['Sentence: 24', '기업', 'NNG', '60'],
 ['Sentence: 24', '임', 'NNG', '60'],
 ['Sentence: 24', '.', 'SF', '60'],
 ['Sentence: 25', '구청', 'NNG', '60'],
 ['Sentence: 25', '관련', 'NNG', '60'],
 ['Sentence: 25', '행사', 'NNG', '60'],
 ['Sentence: 25', '대행사', 'NNG', '60'],
 ['Sentence: 25', '로', 'JKM', '60'],
 ['Sentence: 25', '보', 'VV', '60'],
 ['Sentence: 25', '면', 'ECE', '60'],
 ['Sentence: 25', '되

 ['Sentence: 34', '알바인', 'NNP', '60'],
 ['Sentence: 34', '만큼', 'NNG', '60'],
 ['Sentence: 34', '정산', 'NNG', '60'],
 ['Sentence: 34', '이', 'JKS', '60'],
 ['Sentence: 34', '확실', 'XR', '60'],
 ['Sentence: 34', '하', 'XSA', '60'],
 ['Sentence: 34', '고', 'ECE', '60'],
 ['Sentence: 34', '꿀', 'NNG', '60'],
 ['Sentence: 34', '알바', 'NNG', '60'],
 ['Sentence: 34', '이기', 'VV', '60'],
 ['Sentence: 34', 'ㄴ', 'ETD', '60'],
 ['Sentence: 34', '하다', 'NNP', '60'],
 ['Sentence: 34', '.', 'SF', '60'],
 ['Sentence: 35', '모두', 'NNG', '60'],
 ['Sentence: 35', '가', 'JKS', '60'],
 ['Sentence: 35', '친절', 'NNG', '60'],
 ['Sentence: 35', '하', 'XSV', '60'],
 ['Sentence: 35', 'ㄴ', 'ETD', '60'],
 ['Sentence: 35', '곳', 'NNG', '60'],
 ['Sentence: 35', '.', 'SF', '60'],
 ['Sentence: 35', '시설', 'NNG', '60'],
 ['Sentence: 35', '이', 'JKS', '60'],
 ['Sentence: 35', '매우', 'MAG', '60'],
 ['Sentence: 35', '깨끗', 'XR', '60'],
 ['Sentence: 35', '하', 'XSA', '60'],
 ['Sentence: 35', 'ㅁ', 'ETN', '60'],
 ['Sentence: 35', '.', 'SF', '

...

 ['Sentence: 263', '이', 'JKS', '50'],
 ['Sentence: 263', '없', 'VA', '50'],
 ['Sentence: 263', '더라도', 'ECD', '50'],
 ['Sentence: 263', '선', 'NNG', '50'],
 ['Sentence: 263', '하', 'XSV', '50'],
 ['Sentence: 263', 'ㄴ', 'ETD', '50'],
 ['Sentence: 263', '목적', 'NNG', '50'],
 ['Sentence: 263', '과', 'JC', '50'],
 ['Sentence: 263', '성실', 'NNG', '50'],
 ['Sentence: 263', '하', 'XSV', '50'],
 ['Sentence: 263', 'ㅁ', 'ETN', '50'],
 ['Sentence: 263', '을', 'JKO', '50'],
 ['Sentence: 263', '지니', 'VV', '50'],
 ['Sentence: 263', '었', 'EPT', '50'],
 ['Sentence: 263', '다면', 'ECE', '50'],
 ['Sentence: 263', '다양', 'NNG', '50'],
 ['Sentence: 263', '하', 'XSV', '50'],
 ['Sentence: 263', 'ㄴ', 'ETD', '50'],
 ['Sentence: 263', '경험', 'NNG', '50'],
 ['Sentence: 263', '을', 'JKO', '50'],
 ['Sentence: 263', '쌓', 'VV', '50'],
 ['Sentence: 263', '을', 'ETD', '50'],
 ['Sentence: 263', '수', 'NNB', '50'],
 ['Sentence: 263', '있', 'VV', '50'],
 ['Sentence: 263', '는', 'ETD', '50'],
 ['Sentence: 263', '곳', 'NNG', '50'],
 ['Senten

 ['Sentence: 272', '협력', 'NNG', '66'],
 ['Sentence: 272', '에', 'JKM', '66'],
 ['Sentence: 272', '소극적', 'NNG', '66'],
 ['Sentence: 272', '.', 'SF', '66'],
 ['Sentence: 273', '은근', 'NNG', '66'],
 ['Sentence: 273', '하', 'XSV', '66'],
 ['Sentence: 273', 'ㄴ', 'ETD', '66'],
 ['Sentence: 273', '사내', 'NNG', '66'],
 ['Sentence: 273', '정치', 'NNG', '66'],
 ['Sentence: 273', '가', 'JKS', '66'],
 ['Sentence: 273', '보이', 'VV', '66'],
 ['Sentence: 273', '는', 'ETD', '66'],
 ['Sentence: 273', '근무', 'NNG', '66'],
 ['Sentence: 273', '환경', 'NNG', '66'],
 ['Sentence: 273', '..', 'SW', '66'],
 ['Sentence: 273', '고지식', 'XR', '66'],
 ['Sentence: 273', '하', 'XSA', '66'],
 ['Sentence: 273', 'ㅁ', 'ETN', '66'],
 ['Sentence: 274', '특수', 'NNG', '66'],
 ['Sentence: 274', '외국어', 'NNG', '66'],
 ['Sentence: 274', '전공자', 'NNG', '66'],
 ['Sentence: 274', '로서', 'JKM', '66'],
 ['Sentence: 274', '검찰', 'NNG', '66'],
 ['Sentence: 274', '수사', 'NNG', '66'],
 ['Sentence: 274', '과정', 'NNG', '66'],
 ['Sentence: 274', '에서', 'JKM', '

 ['Sentence: 280', '공기업', 'NNG', '66'],
 ['Sentence: 280', '(', 'SS', '66'],
 ['Sentence: 280', '공무원', 'NNG', '66'],
 ['Sentence: 280', ')', 'SS', '66'],
 ['Sentence: 280', '의', 'NNG', '66'],
 ['Sentence: 280', '장', 'NNG', '66'],
 ['Sentence: 280', '/', 'SP', '66'],
 ['Sentence: 280', '단점', 'NNG', '66'],
 ['Sentence: 280', '을', 'JKO', '66'],
 ['Sentence: 280', '모두', 'MAG', '66'],
 ['Sentence: 280', '가지', 'VV', '66'],
 ['Sentence: 280', '고', 'ECE', '66'],
 ['Sentence: 280', '있', 'VXV', '66'],
 ['Sentence: 280', '는', 'ETD', '66'],
 ['Sentence: 280', '중기업', 'NNG', '66'],
 ['Sentence: 280', '.', 'SF', '66'],
 ['Sentence: 281', '무난', 'XR', '66'],
 ['Sentence: 281', '하', 'XSA', '66'],
 ['Sentence: 281', '게', 'ECD', '66'],
 ['Sentence: 281', '워', 'UN', '66'],
 ['Sentence: 281', '라벨', 'NNG', '66'],
 ['Sentence: 281', '지키', 'VV', '66'],
 ['Sentence: 281', '면서', 'ECE', '66'],
 ['Sentence: 281', '일하', 'VV', '66'],
 ['Sentence: 281', 'ㄹ', 'ETD', '66'],
 ['Sentence: 281', '수', 'NNB', '66'],
 ['Sent

 ['Sentence: 289', '도', 'JX', '66'],
 ['Sentence: 289', '좋', 'VA', '66'],
 ['Sentence: 289', '은', 'ETD', '66'],
 ['Sentence: 289', '편', 'NNB', '66'],
 ['Sentence: 289', '이', 'VCP', '66'],
 ['Sentence: 289', 'ㄴ', 'ETD', '66'],
 ['Sentence: 289', '것', 'NNB', '66'],
 ['Sentence: 289', '같', 'VA', '66'],
 ['Sentence: 289', '고', 'ECE', '66'],
 ['Sentence: 289', '구내', 'NNG', '66'],
 ['Sentence: 289', '식당', 'NNG', '66'],
 ['Sentence: 289', '도', 'JX', '66'],
 ['Sentence: 289', '저', 'NP', '66'],
 ['Sentence: 289', '는', 'JX', '66'],
 ['Sentence: 289', '맛있', 'VA', '66'],
 ['Sentence: 289', '게', 'ECD', '66'],
 ['Sentence: 289', '먹', 'VV', '66'],
 ['Sentence: 289', '었', 'EPT', '66'],
 ['Sentence: 289', '어요', 'EFN', '66'],
 ['Sentence: 289', '급식', 'NNG', '66'],
 ['Sentence: 289', '실', 'NNG', '66'],
 ['Sentence: 289', '화이', 'XR', '66'],
 ['Sentence: 289', '팅', 'UN', '66'],
 ['Sentence: 290', '그냥', 'MAG', '66'],
 ['Sentence: 290', '.', 'SF', '66'],
 ['Sentence: 290', '좋', 'VA', '66'],
 ['Sentence: 290'

 ['Sentence: 296', 'ㄴ', 'ETD', '58'],
 ['Sentence: 296', '하지만', 'MAC', '58'],
 ['Sentence: 296', '실제', 'NNG', '58'],
 ['Sentence: 296', '론', 'XSN', '58'],
 ['Sentence: 296', '주먹구구식', 'NNG', '58'],
 ['Sentence: 296', '채용', 'NNG', '58'],
 ['Sentence: 296', '으로', 'JKM', '58'],
 ['Sentence: 296', '이직률', 'NNG', '58'],
 ['Sentence: 296', '높', 'VA', '58'],
 ['Sentence: 296', '음', 'ETN', '58'],
 ['Sentence: 297', '학교', 'NNG', '58'],
 ['Sentence: 297', '밖', 'NNG', '58'],
 ['Sentence: 297', '청소년', 'NNG', '58'],
 ['Sentence: 297', '지원', 'NNG', '58'],
 ['Sentence: 297', '의', 'JKG', '58'],
 ['Sentence: 297', '특수성', 'NNG', '58'],
 ['Sentence: 297', '있', 'VV', '58'],
 ['Sentence: 297', '으나', 'ECE', '58'],
 ['Sentence: 297', '직원', 'NNG', '58'],
 ['Sentence: 297', '의', 'JKG', '58'],
 ['Sentence: 297', '업무', 'NNG', '58'],
 ['Sentence: 297', '환경', 'NNG', '58'],
 ['Sentence: 297', ',', 'SP', '58'],
 ['Sentence: 297', '복지', 'NNG', '58'],
 ['Sentence: 297', '에', 'JKM', '58'],
 ['Sentence: 297', '한계', 'NNG',

In [80]:
# 불용어 제거
# 조사 제거 

text2_del_SW = []
for t in text2_sent : 
    if ( (t[2][0] !='E') & (t[2][0] !='J') & (t[2][0] !='S') ):
        text2_del_SW.append(t)

In [86]:
words_dict = {} # key: value.
for word in words:
    if word in words_dict:
        words_dict[word] += 1
    else:
        words_dict[word] = 1
        
print(words_dict)

{'안정적': 17, '꾸준히': 1, '오래': 7, '다니': 27, '수': 90, '있': 145, '회사': 31, '그러나': 3, '조금': 4, '고이': 2, '워라밸': 3, '끝판': 1, '왕': 1, '경험': 30, '하': 315, '미국': 1, '문화': 12, '익숙': 1, '않': 34, '위': 4, '올라가': 1, '어렵': 7, '곳': 86, '일': 36, '많': 42, '열심히': 4, '안하': 2, '직원': 27, '친절': 3, '배우': 11, '만하': 5, '하지만': 11, '민': 2, '원인': 1, '때': 8, '밉': 1, '공기': 1, '좋': 67, '선진국': 1, '이': 130, '호주': 1, '수준': 5, '높': 9, '도시인': 1, '캔버라': 1, '5': 3, '시': 1, '칼': 3, '퇴': 2, '가능': 8, '직장': 19, '중앙': 2, '부처': 4, '중': 9, '그리하': 2, '편안': 4, '조직': 20, '분위기': 28, '30': 2, '년': 5, '전': 1, '뒤쳐지': 1, '업무': 60, '프로세스': 1, '왜': 2, '공무원': 21, '지방': 11, '공기업': 16, '정년': 8, '보장': 12, '되': 38, '복지': 20, '포인트': 1, '지급': 1, '노력': 1, '취업': 2, '만큼': 7, '보람': 2, '공공': 15, '기관': 33, '경찰관': 1, '행정관': 1, '무기': 2, '계약': 20, '직': 22, '이루어지': 2, '대략': 1, '680': 1, '여': 2, '명': 2, '근무': 34, '중이': 1, '내근': 2, '외근': 2, '계속': 2, '소통': 1, '놓치': 1, '수평적': 4, '들': 63, '일하': 34, '형성': 2, '퇴근': 4, '적': 13, '급여': 15, '껍데기': 1, '이상': 3, '자부심': 5, 

In [87]:
words_df = pd.DataFrame(words_dict, index = [1]) # dic을 이용하여 데이터프레임 생성

words_df = words_df.transpose() # df 행열 전환
#df.columns = ['f'] # df의 열 이름을 'f'로 지정
words_df = words_df.sort_values(by = words_df.columns[0], ascending = False) # 'f'열 값에 따라 내림차순 정렬

In [88]:
words_df

,1
하,315
있,145
이,130
수,90
곳,86
...,...
삽질,1
일구,1
욕,1
받이,1


In [91]:
text2_del_SW2

[['Sentence: 1', '안정적', 'NNG', '80'],
 ['Sentence: 1', '꾸준히', 'MAG', '80'],
 ['Sentence: 1', '오래', 'MAG', '80'],
 ['Sentence: 1', '다니', 'VV', '80'],
 ['Sentence: 1', '수', 'NNB', '80'],
 ['Sentence: 1', '있', 'VV', '80'],
 ['Sentence: 1', '회사', 'NNG', '80'],
 ['Sentence: 1', '그러나', 'MAC', '80'],
 ['Sentence: 1', '조금', 'MAG', '80'],
 ['Sentence: 1', '고이', 'VV', '80'],
 ['Sentence: 1', '있', 'VXV', '80'],
 ['Sentence: 1', '회사', 'NNG', '80'],
 ['Sentence: 2', '워라밸', 'UN', '80'],
 ['Sentence: 2', '끝판', 'NNG', '80'],
 ['Sentence: 2', '왕', 'NNG', '80'],
 ['Sentence: 2', '경험', 'NNG', '80'],
 ['Sentence: 2', '하', 'XSV', '80'],
 ['Sentence: 2', '수', 'NNB', '80'],
 ['Sentence: 2', '있', 'VV', '80'],
 ['Sentence: 2', '미국', 'NNP', '80'],
 ['Sentence: 2', '문화', 'NNG', '80'],
 ['Sentence: 2', '익숙', 'XR', '80'],
 ['Sentence: 2', '하', 'XSA', '80'],
 ['Sentence: 2', '않', 'VXV', '80'],
 ['Sentence: 2', '위', 'NNG', '80'],
 ['Sentence: 2', '올라가', 'VV', '80'],
 ['Sentence: 2', '어렵', 'VA', '80'],
 ['Sentence: 2

In [90]:
# (수동)불용어 사전 만들기
stopwords=['하', '수', '있', '이','것', '되', '등', '적', '들', '성']
text2_del_SW2 = []
for t in text2_del_SW: 
    if t not in stopwords:
        text2_del_SW2.append(t)

In [ ]:
data.columns = ['Sentence: #', 'word', 'POS', 'score']

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data['word length'] = data['word'].apply(len)
data.head(10)

In [ ]:
sns.boxplot(x='POS', y='score', data=data)

In [178]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor

In [ ]:
import sys
sys.stdout = open('trainingdata.txt', 'w')

for i in test2_text1_sent:
    print(i)
sys.stdout.close()

In [ ]:
import sys

sys.stdout = open('abc.txt', 'w')

for i in range(10):
    print(i)

sys.stdout.close()

In [187]:
traing_text2 = '\n'.join(text2)
traing_text2

'정년보장, 칼퇴근, 적은급여, 껍데기는 공기업 이상하게 직원들이 공기업 다닌다고 자부심 가지고 있음 중소기업급임\n팀바이팀, 자부심은 많지만 월급은 적고 브레인은 많지만 인재를 제대로 활용하지 못하는 곳.\n안정적인 직장이나 그만큼 폐쇄적인 조직\n최소한의 사회적 관계에서 본인 업무를 할 수 있는 공무원\n부대에서 근무할 때는 편한 줄 모름. 하지만 사회에 나오면 이 곳보다 편한 곳이 없음. 야근, 회식 강요 없고 칼퇴근 가능. 개인주의라 업무만 잘 하면 그 외의 것으로 상사 눈치도 별로 안봐도 됨.\n서울 고정 괜찮은 급여 미친듯한 워라밸 한량들이 다니기 딱좋은 회사.\n극지 방문이라는 평생 해보기 힘든 경험을 할 기회가 있음.\n노동자의 권리 향상을 위해 힘쓰는 곳\n관광업계 종사자라면 한국 관광의 주요 사업에 대해 파악할 수 있고, 공기업의 특성상 공무원에 준하는 업무 안정성을 확보할 수 있음.\n제주도에서는 그나마 가장 먹고 살기에 적합한 직업이라고 생각이 된다. 안정된 급여, 복지 등 다양한 혜택이 있다.\n워라밸 지키기에 최고의 직장. 여자들이 다니기에 좋은 직장.\n서로 존중해줍니다.  파트 강사였는데도 적응을 잘 도와줍니다. 수평적인 문화입니다.\n할거 다 하면 자기 공부 할 수 있고 전체적으로 자유로운 분위기에 업무 강도 낮습니다\n지금은 없어진 기관이다. 서울에 있다가 용인으로 옮겨짐.\n연봉이 적고 윗직급이 별로이지만 복지랑 워라밸 생각한다면 무난하게 다닐 수 있다\n일하고자하는 사람이 별로 없고 인사이동이 잦기 때문에 어떤 한 분야를 제대로 전문적으로 아는 사람이 없습니다몇몇 팀장은 결재봇에 불과합니다\n좋은사람은 한없이 좋고 나쁜사람은 한없이 나쁜 곳, 재단 내 정규직과 비정규직의 양극화. 다양한 사람들이 한데 어우러진 동종업계 대기업 취급. 선진적인 부분이 분명 존재함.\n좋은 사람이 많지만, 직원 개별 전문성이 발현되기 어려움.\n환경부 소속기관. 기후변화 정책 관련 연구 진행\n국내유일의 친환경농산물 전문공기업 국내최초 친환경 특

In [188]:
corpus = DoublespaceLineCorpus('trainingdata.txt')
len(corpus)

local variable 'f' referenced before assignment


ValueError: __len__() should return >= 0

In [ ]:
from soynlp.word import WordExtractor
word_extractor